# 4-1 Attrition Trends and Patterns

In [ ]:
import pandas as pd
HRDB_cleaned = pd.read_csv('HRDB_cleaned.csv') 
HRDB_cleaned

,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,...,YearsSinceLastPromotion,YearsWithCurrManager,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField
0,2,3,Male,61,2,2,Research Scientist,2,Married,5130,...,1,7,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences
1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,...,2,2,27,0,Travel_Rarely,591,Research & Development,2,1,Medical
2,8,4,Male,79,3,1,Laboratory Technician,4,Single,3068,...,3,6,32,0,Travel_Frequently,1005,Research & Development,2,2,Life Sciences
3,11,4,Male,67,3,1,Laboratory Technician,3,Divorced,2693,...,0,0,30,0,Travel_Rarely,1358,Research & Development,24,1,Life Sciences
4,12,4,Male,44,2,3,Manufacturing Director,3,Single,9526,...,1,8,38,0,Travel_Frequently,216,Research & Development,23,3,Life Sciences
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,1939,1,Female,47,4,1,Research Scientist,3,Single,2432,...,0,3,32,1,Travel_Frequently,238,Research & Development,5,2,Life Sciences
1466,1967,1,Male,63,3,2,Sales Executive,4,Married,5617,...,0,8,31,1,Travel_Frequently,754,Sales,26,4,Marketing
1467,1968,1,Male,66,3,3,Sales Executive,1,Single,10448,...,2,2,53,1,Travel_Rarely,1168,Sales,24,4,Life Sciences
1468,2027,1,Male,36,3,1,Research Scientist,4,Married,4787,...,2,2,29,1,Travel_Rarely,1092,Research & Development,1,4,Medical


## Top Sub-Categories by Attrition Rate

In [ ]:
# Gives attrition rate of one sub-category:
# Columns = a list of columns we want to get calculations for their sub-categories 
# (for ex., if we write ['Gender' we'll get results for female,male])
# MinEmployeed = filters out groups with less employees.
#RowstoPresent = shows top X results.

def AttritionRateCat(Columns, MinEmployees, RowstoPresent):
    rows = []

    for Column in Columns:
        Categories = HRDB_cleaned[Column].unique().tolist()

        for cat in Categories:
            mask = HRDB_cleaned[HRDB_cleaned[Column]== cat]
            count = len(mask)
            if count < MinEmployees:
                continue
        
            rate = round(100*(mask['Attrition'] == 1).mean(),2)
        
            rows.append({
                'Category': Column,
                'Sub-Category': cat,
                'NumberofEmployees': count,
                'AttritionRate': rate
            })

    return pd.DataFrame(rows).sort_values(by = 'AttritionRate',ascending = False).head(RowstoPresent)




In [ ]:
# This variable returns a list of all columns' headers that are categorial or an object
AllCols = HRDB_cleaned.select_dtypes(include=['object', 'category']).columns.tolist()

AttritionRateCat(AllCols, 10, 5)

,Category,Sub-Category,NumberofEmployees,AttritionRate
9,JobRole,Sales Representative,83,39.76
25,EducationField,Human Resources,27,25.93
12,MaritalStatus,Single,470,25.53
14,BusinessTravel,Travel_Frequently,277,24.91
24,EducationField,Technical Degree,132,24.24


## Top Sub-Categories Pair by Attrition Rate

In [ ]:
# Variation on AttritionRateCat.
# Calculates attrition rate of pairs of sub-categories in 2 columns (for ex. male & in HR)

from itertools import combinations
import pandas as pd

def AttritionRateCross(Columns, MinEmployees, RowstoPresent):
    rows = []

    # 1. Loop through all pairs of columns
    for col1, col2 in combinations(Columns, 2):
        cats1 = HRDB_cleaned[col1].unique().tolist()
        cats2 = HRDB_cleaned[col2].unique().tolist()

        # 2. Loop through subcategories of both columns
        for cat1 in cats1:
            for cat2 in cats2:
                subset = HRDB_cleaned[(HRDB_cleaned[col1] == cat1) & (HRDB_cleaned[col2] == cat2)]
                count = len(subset)

                if count < MinEmployees:
                    continue

                rate = round(100 * (subset['Attrition']).mean(), 2)

                rows.append({
                    'Category1': col1,
                    'Sub-Category1': cat1,
                    'Category2': col2,
                    'Sub-Category2': cat2,
                    'NumberOfEmployees': count,
                    'AttritionRate': rate
                })

    # Build DF
    df = pd.DataFrame(rows)

    if df.empty:
        return df

    # 3. Create merged columns
    df['Sub Category 1'] = df['Category1'] + " = " + df['Sub-Category1'].astype(str)
    df['Sub Category 2'] = df['Category2'] + " = " + df['Sub-Category2'].astype(str)

    # 4. Drop the original 4 columns
    df = df.drop(columns=['Category1', 'Sub-Category1', 'Category2', 'Sub-Category2'])

    # 5. Sort + limit results
    df = df.sort_values(by='AttritionRate', ascending=False).head(RowstoPresent)

    # 6. Reorder columns left → right
    df = df[['Sub Category 1', 'Sub Category 2', 'NumberOfEmployees', 'AttritionRate']]

    return df


In [ ]:
AllCols = HRDB_cleaned.select_dtypes(include=['object']).columns.tolist()
AttritionRateCross(AllCols, 10, 50)

,Sub Category 1,Sub Category 2,NumberOfEmployees,AttritionRate
94,JobRole = Sales Representative,BusinessTravel = Travel_Frequently,23,65.22
69,JobRole = Sales Representative,MaritalStatus = Single,38,55.26
135,JobRole = Sales Representative,EducationField = Marketing,23,52.17
186,BusinessTravel = Travel_Frequently,EducationField = Technical Degree,21,47.62
136,JobRole = Sales Representative,EducationField = Technical Degree,13,46.15
168,MaritalStatus = Single,EducationField = Technical Degree,45,42.22
16,Gender = Female,JobRole = Sales Representative,38,42.11
116,JobRole = Laboratory Technician,EducationField = Technical Degree,19,42.11
167,MaritalStatus = Single,EducationField = Marketing,48,41.67
96,JobRole = Human Resources,BusinessTravel = Travel_Frequently,10,40.00


## Comparison between one sub-category and another category 
## by Attrition Rate

In [ ]:
def AttritionRateByBase(base_column, base_value, other_column, MinEmployees, RowstoPresent=None):
    rows = []

    # 1. Filter to the base subgroup (e.g. JobRole == 'Sales Representative')
    base_subset = HRDB_cleaned[HRDB_cleaned[base_column] == base_value]

    # 2. Get all subcategories of the "other" column within this base group
    categories = base_subset[other_column].unique().tolist()

    for cat in categories:
        # 3. Filter within the base group by the other column
        subset = base_subset[base_subset[other_column] == cat]
        count = len(subset)

        if count < MinEmployees:
            continue

        # 4. Attrition rate inside this intersection
        rate = round(100 * (subset['Attrition'] == 1).mean(), 2)

        # 👉 5. Dynamic formatted label for Sub Category 2
        subcat2_label = f"{other_column} - {cat}"

        rows.append({
            'BaseSubCategory1': f"{base_column} - {base_value}",
            'Sub-Category2': subcat2_label,
            'NumberOfEmployees': count,
            'AttritionRate': rate
        })

    df = pd.DataFrame(rows)

    if df.empty:
        return df

    df = df.sort_values(by='AttritionRate', ascending=False)

    if RowstoPresent is not None:
        df = df.head(RowstoPresent)

    return df

In [ ]:
# Choose sub-category of interest: Sales Representative under Jobrole
# Choose category to compare: JobSatisfaction
# Set Minimal number of employees in intersecting group: 10
# Set maximal value of top rows to present: 50
AttritionRateByBase('JobRole', 'Sales Representative', 'JobSatisfaction', 10, RowstoPresent=50)

,BaseSubCategory1,Sub-Category2,NumberOfEmployees,AttritionRate
1,JobRole - Sales Representative,JobSatisfaction - 1,12,58.33
3,JobRole - Sales Representative,JobSatisfaction - 2,21,47.62
2,JobRole - Sales Representative,JobSatisfaction - 3,27,33.33
0,JobRole - Sales Representative,JobSatisfaction - 4,23,30.43


In [ ]:
AttritionRateByBase('JobRole', 'Sales Representative', 'Gender', 10, RowstoPresent=50)

,BaseSubCategory1,Sub-Category2,NumberOfEmployees,AttritionRate
0,JobRole - Sales Representative,Gender - Female,38,42.11
1,JobRole - Sales Representative,Gender - Male,45,37.78
